In [1]:
import torch as th

# 1 Multiple views of a storage

In [76]:
A = th.ones(13,13)
A

tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

In [77]:
[ (A[i,:].fill_(2), A[:,i].fill_(2)) for i in [1,6,11] ]
[ (A[i+7:i+7-2,i:i+2].fill_(3), A[i:i+2,i-13:i-13+2].fill_(3)) for i in [3, 8] ] 
A

tensor([[1., 2., 1., 1., 1., 1., 2., 1., 1., 1., 1., 2., 1.],
        [2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
        [1., 2., 1., 1., 1., 1., 2., 1., 1., 1., 1., 2., 1.],
        [1., 2., 1., 3., 3., 1., 2., 1., 1., 1., 1., 2., 1.],
        [1., 2., 1., 3., 3., 1., 2., 1., 1., 1., 1., 2., 1.],
        [1., 2., 1., 1., 1., 1., 2., 1., 1., 1., 1., 2., 1.],
        [2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
        [1., 2., 1., 1., 1., 1., 2., 1., 1., 1., 1., 2., 1.],
        [1., 2., 1., 1., 1., 1., 2., 1., 3., 3., 1., 2., 1.],
        [1., 2., 1., 1., 1., 1., 2., 1., 3., 3., 1., 2., 1.],
        [1., 2., 1., 1., 1., 1., 2., 1., 1., 1., 1., 2., 1.],
        [2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
        [1., 2., 1., 1., 1., 1., 2., 1., 1., 1., 1., 2., 1.]])

<b>Note:</b> I didn't loop through the matrix, I just choosed index factors from a loop.

# 2 Eigendecomposition

In [93]:
M = th.randn(20,20)
M_inv = th.inverse(M)
D = th.diag(th.arange(1.,20+1))

In [96]:
A = M_inv.mm(D.mm(M))

In [97]:
eVal, eVec = th.eig(A)

In [98]:
eVal

tensor([[ 1.0000,  0.0000],
        [20.0000,  0.0000],
        [ 2.0000,  0.0000],
        [19.0000,  0.0000],
        [ 3.0000,  0.0000],
        [ 4.0000,  0.0000],
        [18.0000,  0.0000],
        [17.0000,  0.0000],
        [ 5.0000,  0.0000],
        [ 6.0000,  0.0000],
        [16.0000,  0.0000],
        [ 7.0000,  0.0000],
        [15.0000,  0.0000],
        [ 8.0000,  0.0000],
        [ 9.0000,  0.0000],
        [14.0000,  0.0000],
        [10.0000,  0.0000],
        [11.0000,  0.0000],
        [13.0000,  0.0000],
        [12.0000,  0.0000]])

In [101]:
eVal.size()

torch.Size([20, 2])

# 3 Flops per second

In [109]:
import time

In [108]:
A = th.rand(5000,5000).normal_(mean=0, std=1)
B = th.rand(5000,5000).normal_(mean=0, std=1)

In [117]:
i = time.perf_counter()
C = th.mm(A, B)
f = time.perf_counter()
print(f'Time is {f-i} sec')

Time is 2.0698137330000463 sec


# 4 Playing with strides

In [181]:
def mat_mul(X):
    a = 1
    Y = X.clone()
    for row in Y:
        for el in row:
            el *= a
        a += 1
    return Y

In [182]:
def mat_mul_fast(X):
    a = th.Tensor([[i] for i in range(1,X.shape[0]+1)])
    Y = X.clone()
    return Y*a

In [183]:
A = th.ones(4,8)*2
A

tensor([[2., 2., 2., 2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2., 2., 2., 2.]])

In [184]:
B = mat_mul(A)
B

tensor([[2., 2., 2., 2., 2., 2., 2., 2.],
        [4., 4., 4., 4., 4., 4., 4., 4.],
        [6., 6., 6., 6., 6., 6., 6., 6.],
        [8., 8., 8., 8., 8., 8., 8., 8.]])

In [185]:
C = mat_mul(A)
C

tensor([[2., 2., 2., 2., 2., 2., 2., 2.],
        [4., 4., 4., 4., 4., 4., 4., 4.],
        [6., 6., 6., 6., 6., 6., 6., 6.],
        [8., 8., 8., 8., 8., 8., 8., 8.]])

In [186]:
A

tensor([[2., 2., 2., 2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2., 2., 2., 2.]])

In [189]:
A = th.rand(1000,400)

### Slow Function

In [192]:
Si = time.perf_counter()
Slow = mat_mul(A)
Sf = time.perf_counter()
S_time = Sf- Si
print(f'Time is {S_time} sec')

Time is 1.992366733999006 sec


### Fast Function

In [193]:
Fi = time.perf_counter()
Fast = mat_mul_fast(A)
Ff = time.perf_counter()
F_time = Ff- Fi
print(f'Time is {F_time} sec')

Time is 0.001532059999590274 sec


In [195]:
print(f'Fast function is [{S_time/F_time}] faster than the Slow function.')

Fast function is [1300.4495480149826] faster than the Slow function.
